In [3]:
import os
import urllib.request

file_name = "news_articles_2025-05-06.csv"
url = "https://raw.githubusercontent.com/albaneoury/ML.Financial-Models/main/news_articles_2025-05-06.csv"

if not os.path.exists(file_name):
    print("Téléchargement du fichier d’analyse sentimentale...")
    urllib.request.urlretrieve(url, file_name)
    print("✅ Fichier téléchargé avec succès.")
else:
    print("📁 Le fichier existe déjà localement.")

Téléchargement du fichier d’analyse sentimentale...
✅ Fichier téléchargé avec succès.


In [4]:
# ----------------------------------------------------------
# Notebook : Décision finale d'investissement
# ----------------------------------------------------------

import pandas as pd
import numpy as np

# Pour affichage clair
pd.set_option('display.max_columns', None)

# ----------------------------------------------------------
# Étape 1 : Charger les résultats d’analyse technique
# ----------------------------------------------------------

# Supposons que le script précédent ait exporté les résultats dans un CSV
# contenant les colonnes : ['ticker', 'technical_score', 'model_pred', 'last_close', 'decision_model']
tech_results = pd.read_csv("resultats_techniques.csv")
print("📈 Résultats de l’analyse technique :")
print(tech_results)

# ----------------------------------------------------------
# Étape 2 : Charger les données d’analyse sentimentale
# ----------------------------------------------------------

news_df = pd.read_csv("news_articles_2025-05-06.csv")
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

# Filtrer pour le ticker concerné
ticker = tech_results.iloc[0]['ticker']
news_df = news_df[news_df['ticker'] == ticker]

# ----------------------------------------------------------
# Étape 3 : Agréger le score de sentiment
# ----------------------------------------------------------

# Moyenne pondérée par score de confiance
avg_sentiment_score = news_df['sentiment_score'].mean()
print(f"\n📰 Score moyen de sentiment pour {ticker} : {avg_sentiment_score:.2f}")

# Score sur 5 (pour homogénéiser avec score technique)
def score_sentiment_normalise(score):
    return (score + 1) * 2.5  # transforme [-1,1] en [0,5]

sentiment_score_5 = score_sentiment_normalise(avg_sentiment_score)
print(f"Note sentimentale normalisée sur 5 : {sentiment_score_5:.2f}")

# ----------------------------------------------------------
# Étape 4 : Fusion des deux analyses
# ----------------------------------------------------------

tech_score = tech_results.iloc[0]['technical_score']
model_pred = tech_results.iloc[0]['model_pred']
last_close = tech_results.iloc[0]['last_close']
decision_model = tech_results.iloc[0]['decision_model']

# Score global
score_global = (tech_score + sentiment_score_5) / 2
print(f"\n🔗 Score global combiné (technique + sentiment) : {score_global:.2f} / 5")

# ----------------------------------------------------------
# Étape 5 : Prise de décision finale
# ----------------------------------------------------------

if score_global > 3 and model_pred > last_close:
    decision_finale = "INVESTIR"
else:
    decision_finale = "NE PAS INVESTIR / VENDRE"

print(f"\n💡 Décision du modèle d'analyse technique : {decision_model}")
print(f"✅ Décision finale combinée : {decision_finale}")

# ----------------------------------------------------------
# Étape 6 : Sauvegarde des résultats
# ----------------------------------------------------------

results = {
    "ticker": ticker,
    "technical_score": tech_score,
    "sentiment_score_5": sentiment_score_5,
    "score_global": score_global,
    "model_pred": model_pred,
    "last_close": last_close,
    "decision_model": decision_model,
    "decision_finale": decision_finale
}

results_df = pd.DataFrame([results])
results_df.to_csv("decision_finale.csv", index=False)
print("\n📁 Résultat sauvegardé dans decision_finale.csv")

📈 Résultats de l’analyse technique :
  ticker  technical_score  model_pred  last_close decision_model
0   META         3.183413  522.655932   517.36261       INVESTIR

📰 Score moyen de sentiment pour META : nan
Note sentimentale normalisée sur 5 : nan

🔗 Score global combiné (technique + sentiment) : nan / 5

💡 Décision du modèle d'analyse technique : INVESTIR
✅ Décision finale combinée : NE PAS INVESTIR / VENDRE

📁 Résultat sauvegardé dans decision_finale.csv
